In [1]:
import logging

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy
import sklearn.preprocessing


In [2]:
%matplotlib inline
%config InlineBackend.figure_format = 'png'
%config InlineBackend.figure_format = 'retina'

In [3]:
items = pd.read_parquet("items.par")
events = pd.read_parquet("events.par")

In [4]:
items = items.rename(columns = {'book_id': 'item_id'})

In [6]:
items["item_id_enc"] = item_encoder.transform(items["item_id"])

NameError: name 'item_encoder' is not defined

In [7]:
events = events.rename(columns = {'book_id': 'item_id'})

In [8]:
als_recommendations_2 = pd.read_parquet("candidates/inference/als_recommendations.parquet")
content_recommendations_2 = pd.read_parquet("candidates/inference/content_recommendations.parquet")

In [9]:
# зададим точку разбиения
train_test_global_time_split_date = pd.to_datetime("2017-08-01").date()

train_test_global_time_split_idx = events["started_at"] < train_test_global_time_split_date
events_train = events[train_test_global_time_split_idx]
events_test = events[~train_test_global_time_split_idx]

# количество пользователей в train и test
users_train = events_train["user_id"].drop_duplicates()
users_test = events_test["user_id"].drop_duplicates()
# количество пользователей, которые есть и в train, и в test
common_users = list(set(users_train) & set(users_test))

print(len(users_train), len(users_test), len(common_users))

428220 123223 120858


In [10]:
# перекодируем идентификаторы пользователей: 
# из имеющихся в последовательность 0, 1, 2, ...
user_encoder = sklearn.preprocessing.LabelEncoder()
user_encoder.fit(events["user_id"])
events_train["user_id_enc"] = user_encoder.transform(events_train["user_id"])
events_test["user_id_enc"] = user_encoder.transform(events_test["user_id"])

# перекодируем идентификаторы объектов: 
# из имеющихся в последовательность 0, 1, 2, ...
item_encoder = sklearn.preprocessing.LabelEncoder()
item_encoder.fit(items["item_id"])
items["item_id_enc"] = item_encoder.transform(items["item_id"])
events_train["item_id_enc"] = item_encoder.transform(events_train["item_id"])
events_test["item_id_enc"] = item_encoder.transform(events_test["item_id"])

/tmp/ipykernel_38359/3293259747.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  events_train["user_id_enc"] = user_encoder.transform(events_train["user_id"])
/tmp/ipykernel_38359/3293259747.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  events_test["user_id_enc"] = user_encoder.transform(events_test["user_id"])
/tmp/ipykernel_38359/3293259747.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

Se

In [11]:
user_item_matrix_train = scipy.sparse.csr_matrix((
    events_train["rating"],
    (events_train['user_id_enc'], events_train['item_id_enc'])),
    dtype=np.int8)

In [12]:
from implicit.als import AlternatingLeastSquares

als_model = AlternatingLeastSquares(factors=50, iterations=50, regularization=0.05, random_state=0)
als_model.fit(user_item_matrix_train)

/home/mle-user/mle_projects/mle-recsys-start/env_recsys_start/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/mle-user/mle_projects/mle-recsys-start/env_recsys_start/lib/python3.10/site-packages/implicit/cpu/als.py:95: RuntimeWarning: OpenBLAS is configured to use 4 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()
100%|██████████| 50/50 [02:51<00:00,  3.43s/it]


In [13]:
# получим энкодированные идентификаторы всех объектов, известных нам из events_train
train_item_ids_enc = events_train['item_id_enc'].unique()

max_similar_items = 10

# получаем списки похожих объектов, используя ранее полученную ALS-модель
# метод similar_items возвращает и сам объект, как наиболее похожий
# этот объект мы позже отфильтруем, но сейчас запросим на 1 больше
similar_items = als_model.similar_items(train_item_ids_enc, N=max_similar_items+1)

In [14]:
# преобразуем полученные списки в табличный формат
sim_item_item_ids_enc = similar_items[0]
sim_item_scores = similar_items[1]

In [15]:
sim_item_scores

array([[1.0000001 , 0.95125335, 0.8540178 , ..., 0.81872267, 0.80396605,
        0.80324185],
       [1.0000001 , 0.9546376 , 0.838513  , ..., 0.68106806, 0.6803669 ,
        0.6547966 ],
       [0.9999998 , 0.9602783 , 0.89107066, ..., 0.8659329 , 0.8564599 ,
        0.85495627],
       ...,
       [0.99999994, 0.62158924, 0.6183891 , ..., 0.57913935, 0.57824594,
        0.57546216],
       [1.        , 0.6781922 , 0.6698133 , ..., 0.6137276 , 0.6074815 ,
        0.6061272 ],
       [1.        , 0.95466363, 0.95251894, ..., 0.9215702 , 0.9214201 ,
        0.9205395 ]], dtype=float32)

In [25]:
similar_items = pd.DataFrame({
    "item_id_enc": train_item_ids_enc,
    "sim_item_id_enc": sim_item_item_ids_enc.tolist(), 
    "score": sim_item_scores.tolist()
    })
similar_items = similar_items.explode(["sim_item_id_enc", "score"], ignore_index=True)

In [28]:
# приводим типы данных
similar_items["sim_item_id_enc"] = similar_items["sim_item_id_enc"].astype('int32')
similar_items["score"] = similar_items["score"].astype("float")

In [32]:
# получаем изначальные идентификаторы
similar_items["item_id_1"] = item_encoder.inverse_transform(similar_items["item_id_enc"])
similar_items["item_id_2"] = item_encoder.inverse_transform(similar_items["sim_item_id_enc"])


In [33]:
similar_items = similar_items.drop(columns=["item_id_enc", "sim_item_id_enc"])

In [34]:
# убираем пары с одинаковыми объектами
similar_items = similar_items.query("item_id_1 != item_id_2")

In [35]:
similar_items[similar_items['item_id_1']==7126]['item_id_2']

10385      7190
10386     24280
10387      1953
10388     58696
10389     38296
10390      2932
10391      7184
10392    387749
10393      7733
10394     30597
Name: item_id_2, dtype: int64

In [36]:
similar_items.to_parquet("similar_items.parquet") 

In [37]:
def print_sim_items(item_id, similar_items):

    item_columns_to_use = ["item_id", "author", "title", "genre_and_votes", "average_rating", "ratings_count"]
    
    item_id_1 = items.query("item_id == @item_id")[item_columns_to_use]
    display(item_id_1)
    
    si = similar_items.query("item_id_1 == @item_id")
    si = si.merge(items[item_columns_to_use].set_index("item_id"), left_on="item_id_2", right_index=True)
    display(si)

In [38]:
print_sim_items(7144, similar_items)

,item_id,author,title,genre_and_votes,average_rating,ratings_count
1909078,7144,"Fyodor Dostoyevsky, David McDuff, Fyodor Dosto...",Crime and Punishment,"{'Classics': 15812, 'Fiction': 8028, 'Cultural...",4.19,390293


,score,item_id_1,item_id_2,author,title,genre_and_votes,average_rating,ratings_count
4665,0.964479,7144,12505,"Fyodor Dostoyevsky, Anna Brailovsky, Constance...",The Idiot,"{'Classics': 4036, 'Fiction': 2576}",4.18,76392
4666,0.953918,7144,12857,"Fyodor Dostoyevsky, Constance Garnett",The Gambler,"{'Classics': 946, 'Fiction': 729, 'Cultural-Ru...",3.88,22024
4667,0.952009,7144,67326,Fyodor Dostoyevsky,Poor Folk,"{'Classics': 320, 'Fiction': 235, 'Literature-...",3.73,4957
4668,0.946847,7144,5508624,Leo Tolstoy,Family Happiness,"{'Classics': 140, 'Fiction': 112, 'Cultural-Ru...",3.85,3337
4669,0.939762,7144,4934,"Fyodor Dostoyevsky, Fyodor Dostoyevsky, Richar...",The Brothers Karamazov,"{'Classics': 7496, 'Fiction': 5491, 'Cultural-...",4.31,158410
4670,0.938018,7144,17877,"Fyodor Dostoyevsky, Constance Garnett",The House of the Dead,"{'Classics': 533, 'Fiction': 441, 'Cultural-Ru...",4.04,8548
4671,0.937007,7144,929782,"Jack London, Andrew Sinclair",Martin Eden,"{'Classics': 435, 'Fiction': 405, 'Literature-...",4.39,13257
4672,0.936360,7144,28382,Nikolai Gogol,Diary of a Madman and Other Stories,"{'Classics': 284, 'Fiction': 243, 'Short Stori...",4.09,6241
4673,0.936320,7144,17690,"Franz Kafka, Max Brod, Willa Muir, Edwin Muir",The Trial,"{'Classics': 4607, 'Fiction': 4173, 'Literatur...",3.98,135862
4674,0.934541,7144,63038,Victor Hugo,The Man Who Laughs,"{'Classics': 352, 'Fiction': 176, 'Cultural-Fr...",4.22,5449
